In [1]:
import google.generativeai as genai
import os
from google.colab import userdata

# Configure API key and initialize model
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('api_key')

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

def detect_booking_topic(input_text):
    """
    Detects whether the input text is about Room Booking, Vehicle Booking, or Leave Management.
    """
    model = genai.GenerativeModel("gemini-1.5-pro")

    prompt = f"""
    Classify the following input into one of these categories: Room Booking, Vehicle Booking, or Leave Management.

    Input: "{input_text}"
    Output the category name only.
    """

    response = model.generate_content(prompt)
    return response.text.strip()

def detect_named_entity(input_text):
    model = genai.GenerativeModel("gemini-1.5-pro")

    prompt = f"""
    Provide named entity of the following input with entity title like "room_name, start_date, end_date, start_time, end_time, number_of_participants, purpose, pickup_place, destination, date, start_portion, end_portion, leave type" etc

    Input: "{input_text}"
    Output the named entity only.
    """

    response = model.generate_content(prompt)
    return response.text.strip()

def request_missing_entities(topic, detected_entities):
    required_entities = {
        "Room Booking": {"must": ["room_name", "start_date", "end_date", "start_time", "end_time"], "optional": ["number_of_participants", "purpose"]},
        "Vehicle Booking": {"must": ["pickup_place", "destination", "date", "start_time", "end_time"], "optional": ["number_of_participants"]},
        "Leave Management": {"must": ["start_date", "end_date", "start_portion", "end_portion"], "optional": ["purpose", "leave type"]}
    }

    missing_entities = {entity for entity in required_entities[topic]["must"] if entity not in detected_entities}
    optional_entities = [entity for entity in required_entities[topic]["optional"] if entity not in detected_entities]

    for entity in missing_entities:
        detected_entities[entity] = input(f"Please provide {entity}: ")

    for entity in optional_entities:
        value = input(f"Would you like to provide {entity}? (leave blank to skip): ").strip()
        if value:
            detected_entities[entity] = value

    return detected_entities

def main():
    while True:
        user_input = input("Enter your request: ")
        topic = detect_booking_topic(user_input)
        ner = detect_named_entity(user_input)
        detected_entities = {entity.split(':')[0].strip(): entity.split(':')[1].strip() for entity in ner.split('\n') if ':' in entity}

        if topic not in ["Room Booking", "Vehicle Booking", "Leave Management"]:
            print("Sorry, I couldn't determine the topic. Please try again.")
            continue

        updated_entities = request_missing_entities(topic, detected_entities)
        print(f"Your {topic.lower()} request is complete: {updated_entities}")

        new_query = input("Would you like to make another booking? (yes/no): ").strip().lower()
        if new_query != "yes":
            print("Thank you for using the booking system!")
            break

if __name__ == "__main__":
    main()

Enter your request: book a car from Farm-gate to Kurmitola for 5 passengers from 5 pm to 7 pm on tomorrow
Please provide start_time: 
Please provide date: 
Please provide pickup_place: 
Please provide end_time: 
Please provide destination: 
Would you like to provide number_of_participants? (leave blank to skip): 
Your vehicle booking request is complete: {'* **pickup_place': '** Farm-gate', '* **destination': '** Kurmitola', '* **number_of_participants': '** 5', '* **start_time': '** 5 pm', '* **end_time': '** 7 pm', '* **date': '** tomorrow', 'start_time': '', 'date': '', 'pickup_place': '', 'end_time': '', 'destination': ''}
Would you like to make another booking? (yes/no): yes
Enter your request: book a room for 10 participants from 6 pm to 8:47 pm on day after tomorrow for central discussion.
Please provide end_date: 5th April
Please provide start_time: 
Please provide start_date: 
Please provide end_time: 
Please provide room_name: Phoenix
Would you like to provide number_of_parti